<a href="https://colab.research.google.com/github/DarylUnix/Data-Related-Learnings-and-Projects/blob/main/Webscraping_NBA_Summer_League_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [25]:
# Initialize the DataFrame outside the loop
df = pd.DataFrame()

league_name = ["NBA-Summer-League", "Orlando-Pro-Summer-League", "Salt-Lake-City-Summer-League", "California-Classic"]
league_id = [
    [16, 17, 7, 5, 3, 8, 14, 26, 27, 30, 34, 37, 39, 43, 47, 49, 53],
    [20, 19, 18, 4, 2, 9, 13, 25, 28, 31, 33, 36],
    [24, 23, 22, 21, 6, 29, 32, 35, 38, 41, 44, 48, 52],
    [40, 42, 46, 50, 51]
]

for i in range(len(league_name)):
    for j in league_id[i]:
        current_page = 1

        while True:
            url = f'https://basketball.realgm.com/nba/summer/{i+1}/{league_name[i]}/{j}/stats/NBA/0/Averages/All/points/All/desc/{current_page}/Summer_League'
            print(f"Fetching URL: {url}")  # Print the URL to verify
            print(f"Current League: {league_name[i]}, League ID: {j}, Current Page: {current_page}")  # Print the current league, ID, and page number

            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')

            elements = soup.find_all('th')
            headers = [element.text for element in elements]

            if df.empty:
                df = pd.DataFrame(columns=headers)
                if "#" in df.columns:
                    df.rename(columns={"#": "Description"}, inplace=True)

            all_rows = []
            column_data = soup.find_all('tr')
            for row in column_data[1:]:
                row_data = row.find_all('td')
                individual_row_data = [data.text.strip() for data in row_data]
                all_rows.append(individual_row_data)

            description = soup.find_all('h2')[0].text.strip()
            for row in all_rows:
                if row:  # Ensure that the row is not empty
                    row[0] = description

            temp_df = pd.DataFrame(all_rows, columns=df.columns)
            df = pd.concat([df, temp_df], ignore_index=True)

            next_page_button = soup.find('a', href=lambda href: href and f"desc/{current_page + 1}/Summer_League" in href)
            if not next_page_button:
                print(f"No more pages for League ID: {j} in {league_name[i]}")
                break
            else:
                current_page += 1
                print(f"Moving to next page: {current_page} for League ID: {j} in {league_name[i]}")

# Print the DataFrame to verify
print(df)

Fetching URL: https://basketball.realgm.com/nba/summer/1/NBA-Summer-League/16/stats/NBA/0/Averages/All/points/All/desc/1/Summer_League
Current League: NBA-Summer-League, League ID: 16, Current Page: 1
Moving to next page: 2 for League ID: 16 in NBA-Summer-League
Fetching URL: https://basketball.realgm.com/nba/summer/1/NBA-Summer-League/16/stats/NBA/0/Averages/All/points/All/desc/2/Summer_League
Current League: NBA-Summer-League, League ID: 16, Current Page: 2
Moving to next page: 3 for League ID: 16 in NBA-Summer-League
Fetching URL: https://basketball.realgm.com/nba/summer/1/NBA-Summer-League/16/stats/NBA/0/Averages/All/points/All/desc/3/Summer_League
Current League: NBA-Summer-League, League ID: 16, Current Page: 3
No more pages for League ID: 16 in NBA-Summer-League
Fetching URL: https://basketball.realgm.com/nba/summer/1/NBA-Summer-League/17/stats/NBA/0/Averages/All/points/All/desc/1/Summer_League
Current League: NBA-Summer-League, League ID: 17, Current Page: 1
Moving to next page

In [26]:
df.to_csv("NBAsummerleague.csv", index=False)